In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pdb
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

In [ ]:
def split_data(X, y, random_state=1):
    X_train = X.sample(15, random_state=random_state).sort_index().copy()
    y_train = y[X.isin(X_train).all(axis=1)]
    
    X_test = X.loc[~X.isin(X_train).all(axis=1),:]
    y_test = y[X.isin(X_test).all(axis=1)]
    
    return X_train, y_train, X_test, y_test

In [ ]:
url = 'https://raw.githubusercontent.com/Lightning-AI/dl-fundamentals/main/unit01-ml-intro/1.6-perceptron-in-python/perceptron_toydata-truncated.txt'
df = pd.read_csv(url, sep='\t')

In [ ]:
X_train, y_train, X_test, y_test = split_data(df.iloc[:,0:2], df.iloc[:,2])
X_train, y_train, X_test, y_test = torch.from_numpy(X_train.values), torch.from_numpy(y_train.values), torch.from_numpy(X_test.values), torch.from_numpy(y_test.values)
X_train, y_train, X_test, y_test = X_train.to(torch.float32), y_train.to(torch.float32), X_test.to(torch.float32), y_test.to(torch.float32)
X_train, X_test = (X_train-X_train.mean(axis=0)) / X_train.std(axis=0), (X_test-X_test.mean(axis=0)) / X_test.std(axis=0)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = torch.nn.Linear(num_features, 1)

    def forward(self, x):
        logits = self.linear(x)
        probs = torch.sigmoid(logits)
        return probs

In [ ]:
torch.manual_seed(1)

In [ ]:
model = LogisticRegression(num_features=2)
x = torch.tensor([1.1, 2.1])

with torch.no_grad():
    proba = model(x)

print(proba)

tensor([0.4033])


In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.float32)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y

    def __len__(self):
        return self.labels.shape[0]

In [ ]:
train_ds = MyDataset(X_train, y_train)

C:\Users\Santeri\AppData\Local\Temp\ipykernel_12364\1770659071.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(X, dtype=torch.float32)
C:\Users\Santeri\AppData\Local\Temp\ipykernel_12364\1770659071.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(y, dtype=torch.float32)


In [ ]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=8,
    shuffle=True
)

In [ ]:
model = LogisticRegression(num_features=2)
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

n_epoch = 4

for epoch in range(n_epoch):
    
    model = model.train()
    for batch_idx, (features,class_labels) in enumerate(train_loader):
        prob = model(features)
        loss = F.binary_cross_entropy(prob, class_labels.view(prob.shape))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{n_epoch:03d}"
        f" | Batch: {batch_idx+1:03d}/{len(train_loader):03d}"
        f" | Loss: {loss:.2f}")

Epoch: 001/004 | Batch: 001/002 | Loss: 0.90
Epoch: 001/004 | Batch: 002/002 | Loss: 0.90
Epoch: 002/004 | Batch: 001/002 | Loss: 0.85
Epoch: 002/004 | Batch: 002/002 | Loss: 0.88
Epoch: 003/004 | Batch: 001/002 | Loss: 0.92
Epoch: 003/004 | Batch: 002/002 | Loss: 0.71
Epoch: 004/004 | Batch: 001/002 | Loss: 0.81
Epoch: 004/004 | Batch: 002/002 | Loss: 0.77


In [ ]:
pred = torch.where(prob>.5,1,0)

In [ ]:
class_labels.view(pred.shape).to(pred.dtype)

tensor([[0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0]])

In [ ]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    
    correct = .0
    total_examples = 0

    for idx, (features, class_labels) in enumerate(dataloader):

        with torch.inference_mode():
            prob = model(features)

        pred = torch.where(prob>.5, 1, 0)
        lab = class_labels.view(pred.shape).to(pred.dtype)

        compare = lab == pred
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

In [ ]:
compute_accuracy(model, train_loader)

tensor(0.4000)